In [17]:
!pip install transformers torch scikit-learn --quiet

In [18]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from sklearn.metrics import classification_report
from tqdm import tqdm
import os

In [19]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from dataloaders import FactCheckDataset, create_dataloaders

In [20]:
model_name = "vinai/phobert-base"
train_path = "../data/vihallu-train.csv"
test_path  = "../data/vihallu-public-test.csv"

In [21]:
batch_size = 8
max_len = 128
epochs = 10
lr = 2e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

train_loader, test_loader, label2id = create_dataloaders(
    train_path, test_path, tokenizer,
    batch_size=batch_size,
    max_len=max_len
)

id2label = {v:k for k,v in label2id.items()}
print("Label mapping:", label2id)

Label mapping: {'no': 0, 'intrinsic': 1, 'extrinsic': 2}


In [23]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [24]:
optimizer = AdamW(model.parameters(), lr=lr)
num_training_steps = epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
from sklearn.metrics import accuracy_score

for epoch in range(epochs):
    print(f"\n===== Epoch {epoch+1}/{epochs} =====")
    model.train()
    total_loss, total_preds, total_labels = 0, [], []

    loop = tqdm(train_loader, desc="Training", leave=True)
    for batch in loop:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        lr_scheduler.step()

        preds = torch.argmax(outputs.logits, dim=-1)
        total_preds.extend(preds.cpu().numpy())
        total_labels.extend(labels.cpu().numpy())

        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    acc = accuracy_score(total_labels, total_preds)
    print(f"Train Loss: {avg_loss:.4f}, Train Acc: {acc:.4f}")



===== Epoch 1/10 =====


Training:   1%|          | 10/875 [00:31<43:51,  3.04s/it, loss=1.13]

In [ ]:
model.eval()
preds, trues = [], []
with torch.no_grad():
    for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids,
                            attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            preds.extend(predictions.cpu().numpy())
            trues.extend(labels.cpu().numpy())

print("Classification Report:")
print(classification_report(trues, preds, target_names=list(label2id.keys())))

In [ ]:
save_dir = "./checkpoints"
os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Model saved to {save_dir}")

In [ ]:
import pandas as pd

def predict_and_save(model, tokenizer, test_path, save_path, label2id, max_len=128, batch_size=8):
    # Load file test
    df_test = pd.read_csv(test_path)

    # Convert id->label mapping
    id2label = {v: k for k, v in label2id.items()}

    # Tokenize test data (không có nhãn)
    encodings = tokenizer(
        df_test["text"].tolist(),
        truncation=True,
        padding=True,
        max_length=max_len,
        return_tensors="pt"
    )

    test_dataset = torch.utils.data.TensorDataset(
        encodings["input_ids"],
        encodings["attention_mask"],
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model.eval()
    preds = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            batch_preds = torch.argmax(logits, dim=-1)
            preds.extend(batch_preds.cpu().numpy())

    # Map predictions to labels
    pred_labels = [id2label[p] for p in preds]

    # Tạo file submit
    submit_df = pd.DataFrame({
        "id": df_test["id"],
        "predict_label": pred_labels
    })
    submit_df.to_csv(save_path, index=False)
    print(f"Submit file saved to {save_path}")
    return submit_df


submit_path = "../result/submit.csv"
submit_df = predict_and_save(model, tokenizer, test_path, submit_path, label2id, max_len, batch_size)

print(submit_df.head())
